In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import matplotlib.pyplot as plt
from keras.layers import LeakyReLU, Activation, Input, Dense, Dropout, Concatenate, BatchNormalization
from keras.models import Model
from keras.regularizers import l1_l2
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
from keras.datasets import mnist
#physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')
#tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

def build_gan(generator, discriminator, name="gan"):
    '''Build the GAN from a generator and a discriminator'''
    yfake = Activation("linear", name="yfake")(discriminator(generator(generator.inputs)))
    yreal = Activation("linear", name="yreal")(discriminator(discriminator.inputs))
    model = Model(generator.inputs + discriminator.inputs, [yfake, yreal], name=name)
    return model
    

In [ ]:
def disc(image_dim, label_dim, layer_dim=1024, reg=lambda: l1_l2(1e-5, 1e-5)):
    '''Discriminator network'''
    x      = (Input(shape=(image_dim,), name='discriminator_input'))
    label  = (Input(shape=(label_dim,), name='discriminator_label'))
    inputs = (Concatenate(name='input_concatenation'))([x, label])
    a = (Dense(layer_dim, name="discriminator_h1", kernel_regularizer=reg()))(inputs)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(int(layer_dim / 2), name="discriminator_h2", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(int(layer_dim / 4), name="discriminator_h3", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(1, name="discriminator_y", kernel_regularizer=reg()))(a)
    a = (Activation('sigmoid'))(a)
    model = Model(inputs=[x, label], outputs=a, name="discriminator")
    return model

In [ ]:
def gen(noise_dim, label_dim, image_dim, layer_dim=1024, activ='tanh', reg=lambda: l1_l2(1e-5, 1e-5)):
    '''Generator network'''
    z      = (Input(shape=(noise_dim,), name='generator_input'))
    label  = (Input(shape=(label_dim,), name='generator_label'))
    inputs = (Concatenate(name='input_concatenation'))([z, label])
    a = (Dense(int(layer_dim / 4), name="generator_h1", kernel_regularizer=reg()))(inputs)
    a = (LeakyReLU(0.2))(a)    # Trick 5
    a = (Dense(int(layer_dim / 2), name="generator_h2", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(layer_dim, name="generator_h3", kernel_regularizer=reg()))(a)
    a = (LeakyReLU(0.2))(a)
    a = (Dense(np.prod(image_dim), name="generator_x_flat", kernel_regularizer=reg()))(a)
    a = (Activation(activ))(a)    
    model = Model(inputs=[z, label], outputs=[a, label], name="generator")
    return model

In [ ]:
def make_trainable(net, val):
    '''Changes the trainable property of a model as a whole and layer by layer'''
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [ ]:
 ------------------------------------------------------------------------------
# Data preparation
# ------------------------------------------------------------------------------
(x_train, l_train), (x_test, l_test) = mnist.load_data()
x_train = np.concatenate((x_train, x_test))
l_train = np.concatenate((l_train, l_test))

# Normalization according to Trick 1
x_train = x_train.reshape(x_train.shape[0], 784)
x_train = (x_train - 127.5) / 127.5
l_train = to_categorical(l_train)


In [ ]:

# ------------------------------------------------------------------------------
# Parameter choice
# ------------------------------------------------------------------------------    
# Dimension of noise to be fed to the generator
noise_dim = 100
# Dimension of images generated
image_dim = 28 * 28
# Dimension of labels
label_dim = 10

batch_size  = 100
num_batches = int(x_train.shape[0] / batch_size)
num_epochs  = 30

In [ ]:
# ------------------------------------------------------------------------------
# Network creation
# ------------------------------------------------------------------------------
# Create generator ((z, l) -> (x, l))
generator = gen(noise_dim, label_dim, image_dim)
adam = Adam(lr=0.0002, beta_1=0.5)
generator.compile(loss='binary_crossentropy', optimizer=adam)    # Trick 9

# Create discriminator ((x, l) -> y)
discriminator = disc(image_dim, label_dim)
discriminator.compile(loss='binary_crossentropy', optimizer='SGD')    # Trick 9

# Build GAN. Note how the discriminator is set to be not trainable since the beginning
make_trainable(discriminator, False)
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# Training
# ------------------------------------------------------------------------------
for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch + 1, num_epochs))
    for index in range(num_batches):
        # Train the discriminator. It looks like training works best if it is trained first on only real data, and then only
        # on fake data, so let's do that. This is Trick 4.
        make_trainable(discriminator, True)
        # Train dicriminator on real data
        batch       = np.random.randint(0, x_train.shape[0], size=batch_size)
        image_batch = x_train[batch]
        label_batch = l_train[batch]
        y_real      = np.ones(batch_size) + 0.2 * np.random.uniform(-1, 1, size=batch_size)    # Label smoothing. Trick 6
        discriminator.train_on_batch([image_batch, label_batch], y_real)
        # Train the discriminator on fake data
        noise_batch      = np.random.normal(0, 1, (batch_size, noise_dim))    # Trick 3
        generated_images = generator.predict([noise_batch, label_batch])
        y_fake           = np.zeros(batch_size) + 0.2 * np.random.uniform(0, 1, size=batch_size)    # Label smoothing
        d_loss = discriminator.train_on_batch(generated_images, y_fake)   # Recall that generated_images already contains the labels
        # Train the generator. We train it through the whole model. There is a very subtle point here. We want to minimize the error
        # of the discriminator, but on the other hand we want to have the generator maximizing the loss of the discriminator (make him
        # not capable of distinguishing which images are real). One way to achieve this is to change the loss function of the generator
        # by some kind of "negative loss", which in practice is implemented by switching the labels of the real and the fake
        # images. Note that when training the discriminator we were doing the assignment real_image->1, fake_image->0, so now
        # we will do real_image->0, fake_image->1. The order of the outputs is [fake, real], as given by build_gan(). This is Trick 2.
        make_trainable(discriminator, False)
        gan_loss = gan.train_on_batch([noise_batch, label_batch, image_batch, label_batch], [y_real, y_fake])

        print(
            "Batch {}/{}: Discriminator loss = {}, GAN loss = {}".format(index + 1, num_batches, d_loss,
                                                                         gan_loss))
# Save weights. Just saving the whole GAN should work as well
generator.save_weights('generator_cGAN.h5')
discriminator.save_weights('discriminator_cGAN.h5')
gan.save_weights('gan_cGAN.h5')

In [ ]:
 Plotting
# ------------------------------------------------------------------------------
plt.figure(figsize=(20, 2))
for i in range(label_dim):
    im = generator.predict([np.random.uniform(-1, 1, (1, noise_dim)), to_categorical(i, label_dim)])[0].reshape((28, 28))
    plt.subplot(1, label_dim, i+1)
    plt.axis('off')
    plt.imshow(im, cmap='Greys_r')
plt.show()